In [ ]:
%cd '/content/drive/MyDrive/optum/csv/'

/content/drive/MyDrive/optum/csv


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('patient-vaccine.csv')

In [ ]:
data = data.drop(["DESCRIPTION"], axis=1)

In [ ]:
#g = set()
for i in range(len(data)):
    if(data.loc[i, "GENDER"] == 'M'):
      data.loc[i, "GENDER"] = 0
    elif(data.loc[i, "GENDER"] == 'F'):
      data.loc[i, "GENDER"] = 1

In [ ]:
for i in range(len(data)):
  age = 2022 - int((data.loc[i, "BIRTHDATE"]).split('-')[0])
  data.loc[i, "BIRTHDATE"] = age

data.rename(columns = {'BIRTHDATE':'AGE'}, inplace = True)

In [ ]:
encoder = LabelEncoder()
data['COUNTY'] = encoder.fit_transform(data['COUNTY'])

data

In [ ]:
encoder = LabelEncoder()
data['PATIENT'] = encoder.fit_transform(data['PATIENT'])

data

In [ ]:
encoder = LabelEncoder()
data['CODE'] = encoder.fit_transform(data['CODE'])

data

In [ ]:
data = data.drop(["PATIENT"], axis=1)

In [ ]:
dict ={}
classes = 22

for i in range(len(data)):
    row = (data.loc[i, "GENDER"], data.loc[i, "COUNTY"], data.loc[i, "AGE"])
    #temp_vec = np.zeros(classes)
    if(row in dict.keys()):
      exist_vec = dict[row]
      exist_vec[data.loc[i, "CODE"]] = 10
    else:
      temp_vec = np.zeros(classes)
      temp_vec[data.loc[i, "CODE"]] = 10
      dict[row] = temp_vec

In [ ]:
list = []

for key in dict.keys():
  feauture = []
  feauture.extend([key[0],key[1],key[2]])
  vec = dict[key]
  feauture.append(vec)
  list.append(feauture)

In [ ]:
df = pd.DataFrame(list, columns=['A', 'B', 'C', 'D'])

In [ ]:
X = df.drop("D",axis=1).values
y = df["D"].values

In [ ]:
master_list = []
for array in y:
    master_list.append(array)

y = np.vstack(master_list)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

In [ ]:
class Vaccine(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_layer = nn.Linear(in_features=kwargs["input_shape"], out_features=40)
        self.decoder_layer = nn.Linear(in_features=40, out_features=22)

    def forward(self, features):
        input = self.encoder_layer(features)
        activation = torch.relu(input)
        output = self.decoder_layer(activation)
        out = torch.relu(output)
        return out

In [ ]:
import torch.optim as optim
epochs = 1000
learning_rate = 1e-3 

In [ ]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Vaccine(input_shape=3).to(device)

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

criterion = nn.MSELoss()

In [ ]:
for epoch in range(epochs):
    loss = 0
    #count = 0
    for i in range(len(X_train)):
        
        batch_features = X_train[i]
        batch_features = batch_features.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(batch_features)
        train_loss = criterion(outputs, y_train[i])

        train_loss.backward()
        optimizer.step()
        loss += train_loss.item()
    
    
    loss = loss / len(X_train)
    
    if(epoch % 10 == 0):
      print("epoch : {}/{}, loss = {:.8f}".format(epoch + 1, epochs, loss))

In [ ]:
test_examples = []
reconstruction = []

with torch.no_grad():
    #count=0
    for i in range(len(X_test)):
        batch_features = X_test[i]
        test_examples.append(y_test[i].numpy())
        batch_features = batch_features.to(device)
        reconstruction.append((model(batch_features).cpu()).numpy())

In [ ]:
with torch.no_grad():
    number = 20
    
    for index in range(number):
        print(test_examples[index])
        print(reconstruction[index])
        print("\n\n")

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0. 10.
  0. 10.  0.  0.]
[0.       0.       0.       0.       0.       0.       0.       0.
 0.       0.       0.       9.588057 0.       0.       0.       0.
 0.       9.592865 0.       0.       0.       0.      ]



[ 0.  0.  0.  0.  0.  0. 10.  0. 10.  0.  0. 10.  0.  0.  0.  0.  0. 10.
  0.  0. 10.  0.]
[0.        0.        0.        0.        0.        0.3713777 0.
 0.        0.        0.        0.        9.643013  0.        0.
 0.        0.        0.        9.632902  0.        0.        0.
 0.       ]



[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0. 10.
  0. 10. 10.  0.]
[0.       0.       0.       0.       0.       0.       0.       0.
 0.       0.       0.       9.574054 0.       0.       0.       0.
 0.       9.609198 0.       0.       0.       0.      ]



[ 0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  0. 10.  0.  0.  0.  0.  0. 10.
  0.  0. 10.  0.]
[0.        0.        0.        0.        0.